In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   0.75364   0.75369  0.75324    0.75324   0.75379   
1 2021-04-01 06:05:00   0.75329   0.75362  0.75315    0.75330   0.75342   
2 2021-04-01 06:10:00   0.75326   0.75363  0.75326    0.75360   0.75340   
3 2021-04-01 06:15:00   0.75364   0.75426  0.75364    0.75426   0.75375   
4 2021-04-01 06:20:00   0.75423   0.75495  0.75407    0.75469   0.75436   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   0.75382  0.75339    0.75339   0.75372   0.75376  0.75332    0.75332   
1   0.75375  0.75327    0.75343   0.75336   0.75368  0.75321    0.75336   
2   0.75377  0.75340    0.75372   0.75333   0.75370  0.75333    0.75366   
3   0.75440  0.75375    0.75440   0.75370   0.75433  0.75370    0.75433   
4   0.75509  0.75421    0.75483   0.75430   0.75502  0.75414    0.75476   

   Volume  
0      63  
1      62  
2      48  
3      46  
4      82  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

def fib_bb(highs, lows, closes, volumes, length=200, mult=3):
    hlc3 = (highs + lows + closes) / 3
    basis = (hlc3 * volumes).rolling(window=length).sum() / volumes.rolling(window=length).sum()
    dev = hlc3.rolling(window=length).std() * mult
    upper236 = basis + (0.236 * dev)
    upper382 = basis + (0.382 * dev)
    upper5 = basis + (0.5 * dev)
    upper618 = basis + (0.618 * dev)
    upper764 = basis + (0.764 * dev)
    upper1 = basis + (1 * dev)
    lower236 = basis - (0.236 * dev)
    lower382 = basis - (0.382 * dev)
    lower5 = basis - (0.5 * dev)
    lower618 = basis - (0.618 * dev)
    lower764 = basis - (0.764 * dev)
    lower1 = basis - (1 * dev)

    return upper236, upper382, upper5, upper618, upper764, upper1, lower236, lower382, lower5, lower618, lower764, lower1


In [7]:
# Add technical indicators (for additional features)
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['upper236'], df['upper382'], df['upper5'], df['upper618'], df['upper764'], df['upper1'], df['lower236'], df['lower382'], df['lower5'], df['lower618'], df['lower764'], df['lower1'] = fib_bb(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], df['Volume'])
df['sar'] = psar(df)
df['rsi'] = rsi(df)
df['rsi_sma'] = df['rsi'].rolling(50).mean()

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-04-01 23:05:00   0.76167   0.76170  0.76156    0.76164   0.76182   
1     2021-04-01 23:10:00   0.76167   0.76176  0.76167    0.76169   0.76182   
2     2021-04-01 23:15:00   0.76166   0.76166  0.76166    0.76166   0.76181   
3     2021-04-01 23:25:00   0.76162   0.76162  0.76144    0.76146   0.76178   
4     2021-04-01 23:30:00   0.76149   0.76165  0.76149    0.76161   0.76167   
...                   ...       ...       ...      ...        ...       ...   
74309 2022-04-01 05:35:00   0.74750   0.74773  0.74747    0.74767   0.74765   
74310 2022-04-01 05:40:00   0.74768   0.74768  0.74740    0.74756   0.74780   
74311 2022-04-01 05:45:00   0.74756   0.74770  0.74754    0.74766   0.74770   
74312 2022-04-01 05:50:00   0.74764   0.74766  0.74734    0.74762   0.74780   
74313 2022-04-01 05:55:00   0.74761   0.74763  0.74726    0.74752   0.74775   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
pair = 'EUR_USD'
rounding = 3 if 'JPY' in pair else 5

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, fib_val, use_sar, use_sar_sl, use_rsi):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None

    for i in range(2, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        macd2, macdsignal2, lower2, lower_sl2, upper2, upper_sl2, mid_low2, mid_high2 = df.loc[df.index[i - 2], ['macd', 'macdsignal', 'lower1', f'lower{fib_val}', 'upper1', f'upper{fib_val}', 'Mid_Low', 'Mid_High']]
        macd1, macdsignal1, lower1, lower_sl1, upper1, upper_sl1, mid_low1, mid_high1, mid_close1, sar1, rsi1, rsi_sma1 = df.loc[df.index[i - 1], ['macd', 'macdsignal', 'lower1', f'lower{fib_val}', 'upper1', f'upper{fib_val}', 'Mid_Low', 'Mid_High', 'Mid_Close', 'sar', 'rsi', 'rsi_sma']]
        spread = abs(curr_ask_open - curr_bid_open)
        macd_vals = [macd2, macdsignal2, macd1, macdsignal1]

        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) < 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) > 0

        touched_lower_band = mid_low2 <= lower2 or mid_low1 <= lower1
        touched_upper_band = mid_high2 >= upper2 or mid_high1 >= upper1

        closed_above_band = mid_close1 > lower1
        closed_below_band = mid_close1 < upper1

        sar_buy_signal = sar1 < mid_low1 if use_sar else True
        sar_sell_signal = sar1 > mid_high1 if use_sar else True

        rsi_buy_signal = rsi1 > rsi_sma1 if use_rsi else True
        rsi_sell_signal = rsi1 < rsi_sma1 if use_rsi else True

        buy_signal = macd_buy_signal and touched_lower_band and closed_above_band and sar_buy_signal and rsi_buy_signal
        sell_signal = macd_sell_signal and touched_upper_band and closed_below_band and sar_sell_signal and rsi_sell_signal

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)
                pips_diff = abs(lower_sl1 - lower1)
                stop_loss = (sar1 - pips_diff) if use_sar_sl else (min(mid_low2, mid_low1) - pips_diff)
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = abs(open_price - stop_loss)

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                pips_diff = abs(upper1 - upper_sl1)
                stop_loss = (sar1 + pips_diff) if use_sar_sl else (max(mid_high2, mid_high1) + pips_diff)
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = abs(stop_loss - open_price)

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2.0]
spread_cutoffs = [0.1]
fib_vals = [236, 382, 5, 618, 764]
use_sar_vals = [True, False]
use_sar_sl_vals = [True, False]
use_rsi_vals = [True, False]

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for fib_val in fib_vals:
            for use_sar in use_sar_vals:
                for use_sar_sl in use_sar_sl_vals:
                    for use_rsi in use_rsi_vals:
                        all_combos.append((risk_reward_ratio, spread_val, fib_val, use_sar, use_sar_sl, use_rsi))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_fib_val = None
best_use_sar_val = None
best_use_sar_sl_val = None
best_use_rsi_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, fib_val, use_sar, use_sar_sl, use_rsi in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, fib_val, use_sar, use_sar_sl, use_rsi)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'fib_val': fib_val, 'use_sar': use_sar, 'use_sar_sl': use_sar_sl, 'use_rsi': use_rsi})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_fib_val = fib_val
        best_use_sar_val = use_sar
        best_use_sar_sl_val = use_sar_sl
        best_use_rsi_val = use_rsi

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 120

-826.4400499999928
Num buys: 38
Num sells: 35
Num trades: 73
Num wins: 23
Num losses: 50
Win streak: 3
Loss streak: 16
Avg pips risked: 0.00208534246575342
Remaining runs: 119
Best reward so far: -826.4400499999928

-521.1725800000069
Num buys: 11
Num sells: 17
Num trades: 28
Num wins: 6
Num losses: 22
Win streak: 2
Loss streak: 12
Avg pips risked: 0.001818214285714298
Remaining runs: 118
Best reward so far: -521.1725800000069

-154.06922999999563
Num buys: 10
Num sells: 11
Num trades: 21
Num wins: 9
Num losses: 12
Win streak: 3
Loss streak: 5
Avg pips risked: 0.002423333333333317
Remaining runs: 117
Best reward so far: -154.06922999999563

-331.4481200000009
Num buys: 18
Num sells: 18
Num trades: 36
Num wins: 10
Num losses: 26
Win streak: 3
Loss streak: 11
Avg pips risked: 0.0021886111111111086
Remaining runs: 116
Best reward so far: -154.06922999999563

-256.1218999999955
Num buys: 11
Num sells: 12
Num trades: 23
Num wins: 9
Num losses: 14
Win streak: 3
Loss streak: 5


In [11]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best fib val: ' + str(best_fib_val))
print('Best use sar val: ' + str(best_use_sar_val))
print('Best use sar sl val: ' + str(best_use_sar_sl_val))
print('Best use rsi val: ' + str(best_use_rsi_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 925.0208300000123
Best risk/reward ratio: 2.0
Best spread: 0.1
Best fib val: 382
Best use sar val: True
Best use sar sl val: False
-----------------------
Top results:
{'reward': 733, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 236, 'use_sar': True, 'use_sar_sl': False}
{'reward': 925, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 382, 'use_sar': True, 'use_sar_sl': False}
{'reward': 662, 'ratio': 1.5, 'spread': 0.1, 'fib_val': 382, 'use_sar': True, 'use_sar_sl': False}
{'reward': 722, 'ratio': 1.5, 'spread': 0.1, 'fib_val': 618, 'use_sar': True, 'use_sar_sl': True}
{'reward': 699, 'ratio': 1.5, 'spread': 0.1, 'fib_val': 5, 'use_sar': True, 'use_sar_sl': True}
{'reward': 822, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 5, 'use_sar': True, 'use_sar_sl': False}
{'reward': 779, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 382, 'use_sar': False, 'use_sar_sl': False}
{'reward': 766, 'ratio': 1.5, 'spread': 0.1, 'fib_val': 236, 'use_sar': True, 'use_sar_sl':

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best fib val: ' + str(best_fib_val))
print('Best use sar val: ' + str(best_use_sar_val))
print('Best use sar sl val: ' + str(best_use_sar_sl_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: -110.10094000001425
Best risk/reward ratio: 1.0
Best spread: 0.1
Best fib val: 764
Best use sar val: True
Best use sar sl val: False
-----------------------
Top results:
{'reward': -379, 'ratio': 1.5, 'spread': 0.1, 'fib_val': 382, 'use_sar': True, 'use_sar_sl': True}
{'reward': -273, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 764, 'use_sar': True, 'use_sar_sl': False}
{'reward': -285, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 382, 'use_sar': True, 'use_sar_sl': True}
{'reward': -166, 'ratio': 1.0, 'spread': 0.1, 'fib_val': 764, 'use_sar': False, 'use_sar_sl': True}
{'reward': -333, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 618, 'use_sar': True, 'use_sar_sl': False}
{'reward': -417, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 236, 'use_sar': False, 'use_sar_sl': True}
{'reward': -283, 'ratio': 2.0, 'spread': 0.1, 'fib_val': 5, 'use_sar': True, 'use_sar_sl': True}
{'reward': -165, 'ratio': 1.0, 'spread': 0.1, 'fib_val': 764, 'use_sar': True, 'us